In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import re
import string
from fuzzywuzzy import fuzz
#from merge_charlotte import parse_address, fuzz_comparisons, append_fuzz_scores
import merge_charlotte as lib

%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
fname_business = '../data/yelp/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json'

In [4]:
with open(fname_business) as f:
    B = pd.DataFrame(json.loads(line) for line in f)

In [4]:
B.describe()

,latitude,longitude,open,review_count,stars
count,61184.000000,61184.000000,61184,61184.000000,61184.000000
mean,37.326077,-97.491803,0.878089,28.272506,3.673305
std,5.741085,29.624473,0.327186,88.652050,0.891207
min,32.871923,-115.386550,False,3.000000,1.000000
25%,33.509601,-115.052224,1,4.000000,3.000000
50%,35.994636,-111.932944,1,8.000000,3.500000
75%,36.240430,-80.863329,1,21.000000,4.500000
max,56.036545,8.549249,True,4578.000000,5.000000


In [5]:
B.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61184 entries, 0 to 61183
Data columns (total 15 columns):
attributes       61184 non-null object
business_id      61184 non-null object
categories       61184 non-null object
city             61184 non-null object
full_address     61184 non-null object
hours            61184 non-null object
latitude         61184 non-null float64
longitude        61184 non-null float64
name             61184 non-null object
neighborhoods    61184 non-null object
open             61184 non-null bool
review_count     61184 non-null int64
stars            61184 non-null float64
state            61184 non-null object
type             61184 non-null object
dtypes: bool(1), float64(3), int64(1), object(10)
memory usage: 7.1+ MB


In [6]:
B.head().T

,0,1,2,3,4
attributes,{u'By Appointment Only': True},"{u'Happy Hour': True, u'Accepts Credit Cards':...",{u'Good for Kids': True},{},"{u'Alcohol': u'full_bar', u'Noise Level': u'av..."
business_id,vcNAWiLM4dR7D2nwwJ7nCA,UsFtqoBl7naz8AVUBZMjQQ,cE27W9VPgO88Qxe4ol6y_g,HZdLhv6COCleJMo7nPl-RA,mVHrayjG3uZ_RLHkLj-AMg
categories,"[Doctors, Health & Medical]",[Nightlife],"[Active Life, Mini Golf, Golf]","[Shopping, Home Services, Internet Service Pro...","[Bars, American (New), Nightlife, Lounges, Res..."
city,Phoenix,Dravosburg,Bethel Park,Pittsburgh,Braddock
full_address,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","202 McClure St\nDravosburg, PA 15034","1530 Hamilton Rd\nBethel Park, PA 15234","301 S Hills Vlg\nPittsburgh, PA 15241","414 Hawkins Ave\nBraddock, PA 15104"
hours,"{u'Thursday': {u'close': u'17:00', u'open': u'...",{},{},"{u'Monday': {u'close': u'21:00', u'open': u'10...","{u'Tuesday': {u'close': u'19:00', u'open': u'1..."
latitude,33.4993,40.3505,40.3569,40.3576,40.4087
longitude,-111.984,-79.8869,-80.0159,-80.06,-79.8664
name,"Eric Goldberg, MD",Clancy's Pub,Cool Springs Golf Center,Verizon Wireless,Emil's Lounge
neighborhoods,[],[],[],[],[]


In [7]:
B.state.value_counts()

AZ     25230
NV     16485
NC      4963
QC      3921
PA      3041
EDH     2971
WI      2307
BW       934
IL       627
ON       351
SC       189
MLN      123
RP        13
ELN       10
FIF        4
CA         3
SCB        3
MN         1
MA         1
KHL        1
XGL        1
WA         1
NW         1
OR         1
NTH        1
HAM        1
Name: state, dtype: int64

In [8]:
B_keep = B[B.state.isin({'AZ','NV','PA','WI','IL','NC'})]

In [9]:
B_keep.groupby('state').describe()[['review_count', 'stars']].unstack(-1)

review_count                                               stars  \
             count       mean         std min 25% 50% 75%   max  count   
state                                                                    
AZ           25230  25.238962   55.871449   3   4   8  21  1512  25230   
IL             627  20.987241   33.924552   3   5   9  22   450    627   
NC            4963  20.651823   40.028150   3   4   8  19   983   4963   
NV           16485  45.672066  150.342727   3   5  10  29  4578  16485   
PA            3041  23.810917   43.332042   3   4   9  24   695   3041   
WI            2307  20.669267   37.064819   3   4   8  20   723   2307   

                                                 
           mean       std min 25%  50%  75% max  
state                                            
AZ     3.684503  0.940502   1   3  4.0  4.5   5  
IL     3.569378  0.897042   1   3  3.5  4.0   5  
NC     3.588858  0.844681   1   3  3.5  4.0   5  
NV     3.658477  0.908426   1   3  3.5  4.5   5  
PA     3.682999  0.829634   1   3  3.5  4.5   5  
WI     3.643693  0.857121   1   3  3.5  4.5   5

In [10]:
B_keep['type'].value_counts()

business    52653
Name: type, dtype: int64

In [685]:
B_rest = B[(B.state.isin({'AZ','NV','PA','WI','IL','NC'})) & \
           (B.categories.apply(lambda x: 'Restaurants' in x))]
save_to_pickle(B_rest, '../data/yelp/yelp_US_restaurants.pkl')

# Merge with Health Inspections

In [2]:
def save_to_pickle(data, fname):
    with open(fname, 'wb') as handle:
        pickle.dump(data, handle)


def open_pickle(f_name):
    with open(f_name, 'rb') as f:
        data = pickle.load(f)
    return data

### Phoenix, AZ

In [626]:
test_AZ = open_pickle('../data/phx/phoenix_yelp_merge.pkl')

In [627]:
test_AZ.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3984 entries, (1130 the restaurant, 455, north 3rd st, phoenix, 85004) to (9990, north 90th st, scottsdale, 85258)
Data columns (total 43 columns):
address                    3984 non-null object
addressy_                  3984 non-null object
attributes                 3984 non-null object
avg_2                      2250 non-null float64
avg_w_2                    2250 non-null float64
business_id                3984 non-null object
categories                 3984 non-null object
city_                      3984 non-null object
complex                    3984 non-null object
complexy_                  3984 non-null object
cutting_edge               3984 non-null object
full_address               3984 non-null object
fuzz_partial_ratio         2250 non-null float64
fuzz_partial_ratio_2       2250 non-null float64
fuzz_ratio                 2250 non-null float64
fuzz_ratio_2               2250 non-null float64
fuzz_token_set_ratio       2250 n

### Madison, WI

In [535]:
import merge_madison as lib

In [600]:
reload(lib)

<module 'merge_madison' from 'merge_madison.py'>

In [617]:
B_WI = B[(B.state=='WI')& (B.categories.apply(lambda x: 'Restaurants' in x))]
B_WI['address'] = B_WI.full_address.apply(lambda x: x.replace('\n',' ')\
                                                     .replace('Ste','Suite')\
                                                     .replace(',','')\
                                                     .replace(' WI ', ' '))

B_address = pd.Series(zip(B_WI.full_address, B_WI.neighborhoods)).apply(lambda x: lib.parse_address(*x))
B_address.set_index(B_WI.index, inplace=True)
col = B_address.columns.values
col[0] = 'city_'
B_address.columns = col
B_WI = pd.concat([B_WI, B_address], axis=1)
B_WI['name_'] = B_WI.name.apply(standard_name)

In [539]:
H = lib.open_pickle('../data/mad/mad_health_FINAL.pkl')
H.head()

,address,inspections,name,type
MainContent_12775-2814,"1405 EMIL ST\nMADISON, WI 53713",{u'MainContent_2462414': {u'date': u'2/13/2014...,Licari's Bar & Grill,Primarily Restaurant
MainContent_1496-974,"119 W MAIN ST\nMADISON, WI 53703",{u'MainContent_2471561': {u'date': u'3/27/2014...,Paradise Lounge,Primarily Restaurant
MainContent_1497-981,"226 STATE ST\nMADISON, WI 53703",{u'MainContent_2231550': {u'date': u'1/18/2012...,Nick's Restaurant,Primarily Restaurant
MainContent_1660-1106,"425 N FRANCES ST\nMADISON, WI 53703",{u'MainContent_2478840': {u'date': u'4/29/2014...,Porta Bella,Primarily Restaurant
MainContent_19803-4575,"1029 S PARK ST\nMADISON, WI 53715",{u'MainContent_2325667': {u'date': u'11/29/201...,Oriental Shop,Retail Food Establishment


In [618]:
WI = H.copy()
WI['name_'] = H.name.apply(lib.standard_name)
WI['id_'] = H.index

WI_address = WI.address.apply(lib.parse_address)
col = WI_address.columns.values
col[0] = 'city_'
WI_address.columns = col

WI = pd.concat([WI, WI_address], axis=1)

WI.head().T

,MainContent_12775-2814,MainContent_1496-974,MainContent_1497-981,MainContent_1660-1106,MainContent_19803-4575
address,"1405 EMIL ST\nMADISON, WI 53713","119 W MAIN ST\nMADISON, WI 53703","226 STATE ST\nMADISON, WI 53703","425 N FRANCES ST\nMADISON, WI 53703","1029 S PARK ST\nMADISON, WI 53715"
inspections,{u'MainContent_2462414': {u'date': u'2/13/2014...,{u'MainContent_2471561': {u'date': u'3/27/2014...,{u'MainContent_2231550': {u'date': u'1/18/2012...,{u'MainContent_2478840': {u'date': u'4/29/2014...,{u'MainContent_2325667': {u'date': u'11/29/201...
name,Licari's Bar & Grill,Paradise Lounge,Nick's Restaurant,Porta Bella,Oriental Shop
type,Primarily Restaurant,Primarily Restaurant,Primarily Restaurant,Primarily Restaurant,Retail Food Establishment
name_,licaris bar grill,paradise lounge,nicks restaurant,porta bella,oriental shop
id_,MainContent_12775-2814,MainContent_1496-974,MainContent_1497-981,MainContent_1660-1106,MainContent_19803-4575
city_,madison,madison,madison,madison,madison
complex,,,,,
num,1405,119,226,425,1029
street,emil st,west main st,state st,north frances st,south park st


In [610]:
WI.ix['MainContent_73719-81991']

address            214 COTTAGE GROVE RD\nCOTTAGE GROVE, WI 53527
inspections    {u'MainContent_2318593': {u'date': u'10/23/201...
name                                                Village Cafe
type                                        Primarily Restaurant
name_                                               village cafe
id_                                      MainContent_73719-81991
city_                                                      grove
complex                                                         
num                                                          214
street                                  cottage grove rd cottage
suite                                                           
zip                                                        53527
Name: MainContent_73719-81991, dtype: object

In [599]:
a = '214 COTTAGE GROVE RD\nCOTTAGE GROVE, WI 53527'.lower()
cities = {'mc farland':'mcfarland', 'de forest':'deforest', 'cottage grove':'cottage_grove', 'sun prairie':'sun_prairie'}
for key, value in cities.iteritems():
    print re.sub(r'\n(%s, wi)\b' % key, '\n%s, wi' % value, a)

214 cottage grove rd
cottage grove, wi 53527
214 cottage grove rd
cottage grove, wi 53527
214 cottage grove rd
cottage grove, wi 53527
214 cottage grove rd
cottage_grove, wi 53527


In [619]:
WI_ind = ['name_','num','street','city_','zip']
#NV_ind = ['n' 'street','city_','zip']
temp = WI.set_index(WI_ind).join(B_WI.set_index(WI_ind), how='inner', rsuffix = 'y_')
print temp.shape

H_x = WI[~WI.id_.isin(temp.id_)]
B_x = B_WI[~B_WI.business_id.isin(temp.business_id)]

WI_ind2 = ['num','street','city_','zip']
temp2 = H_x.set_index(WI_ind2).join(B_x.set_index(WI_ind2), how='inner', rsuffix = 'y_')

print temp2.shape

temp_fuzz = lib.append_fuzz_scores(temp2)

(298, 25)
(513, 27)


In [620]:
ind_A = (temp_fuzz['max'] >= 75) & (temp_fuzz.avg_w_2 >= 75)
ind_B = (temp_fuzz['max'] >= 60) & (temp_fuzz.avg_w_2 >= 60) & (temp_fuzz.avg_w_3 >= 80)
WI_level2 = pd.concat([temp, temp_fuzz[ind_A | ind_B]])

print WI_level2.shape

H_x2 = WI[~WI.id_.isin(WI_level2.id_)]
B_x2 = B_WI[~B_WI.business_id.isin(WI_level2.business_id)]

(568, 50)


In [621]:
WI_ind3 = ['city_','zip']
temp3 = H_x2.set_index(WI_ind3).join(B_x2.set_index(WI_ind3), how='inner', rsuffix = 'y_')
print temp3.shape

temp_fuzz3 = lib.append_fuzz_scores(temp3)
print temp_fuzz3.shape

(42492, 31)
(42492, 54)


In [622]:
ind_A3 = (temp_fuzz3['max'] >= 75) & (temp_fuzz3.avg_w_2 >= 75)
ind_B3 = (temp_fuzz3['max'] >= 60) & (temp_fuzz3.avg_w_2 >= 60) & (temp_fuzz3.avg_w_3 >= 80)
temp_fuzz3[ind_A3 | ind_B3].to_csv('../data/mad/merge_dump_11.csv', encoding='utf-8')

In [623]:
def to_int(x):
    try:
        out = int(x)
    except:
        out = np.nan
    return out

ind_A3 = (temp_fuzz3['max'] >= 75) & (temp_fuzz3.avg_w_2 >= 75)
ind_B3 = (temp_fuzz3['max'] >= 60) & (temp_fuzz3.avg_w_2 >= 60) & (temp_fuzz3.avg_w_3 >= 80)
ind_C3 = temp_fuzz3.num == temp_fuzz3.numy_
ind_D3 = (temp_fuzz3.street == temp_fuzz3.street) & \
         (abs(temp_fuzz3.num.apply(to_int) - temp_fuzz3.numy_.apply(to_int)) <= 100)
WI_level3 = pd.concat([WI_level2, temp_fuzz3[(ind_A3 | ind_B3) & (ind_C3 | ind_D3)]])
print WI_level3.shape

(654, 54)


In [625]:
save_to_pickle(WI_level3,'../data/mad/madison_yelp_merge.pkl')

In [596]:
temp2.to_csv('../data/mad/merge_dump_01.csv', encoding='utf-8')

In [624]:
temp_fuzz.to_csv('../data/mad/merge_dump_03.csv', encoding='utf-8')

In [550]:
b_list = {'MainContent_73157-81352','MainContent_73211-81413','MainContent_LICFDD-2013-00052',
          'MainContent_LICFDD-2015-00213','MainContent_73488-81733','MainContent_76218-87721',
          'MainContent_73048-81214','MainContent_LICFDD-2015-00295'}
ind_b_list = WI.id_.isin(b_list)

In [563]:
reload(lib)

<module 'merge_madison' from 'merge_madison.py'>

In [582]:
B_WI.city_.value_counts()

madison      719
middleton     68
prairie       45
fitchburg     42
monona        34
verona        21
waunakee      14
farland        6
grove          5
deforest       5
forest         5
mcfarland      4
windsor        3
dane           2
stoughton      2
Name: city_, dtype: int64

### Las Vegas, NV

In [626]:
import merge_vegas as lib

In [671]:
reload(lib)

<module 'merge_vegas' from 'merge_vegas.py'>

In [672]:
test = B_NV[B_NV.business_id=='F14WDg0i6yZwXbflJ8ndaw']
print test.full_address.values
pd.Series(zip(test.full_address, test.neighborhoods)).apply(lambda x: lib.parse_address(*x))

[ u'Caesars Palace Hotel & Casino\n3570 Las Vegas Blvd S\nThe Strip\nLas Vegas, NV 89109']


,city,complex,num,street,suite,zip
0,las_vegas,caesars palace hotel casino,3570,las vegas blvd south,,89109


In [673]:
B_NV = B[(B.state=='NV')& (B.categories.apply(lambda x: 'Restaurants' in x))]
B_NV['address'] = B_NV.full_address.apply(lambda x: x.replace('\n',' ')\
                                                     .replace('Ste','Suite')\
                                                     .replace(',','')\
                                                     .replace(' NV ', ' '))
B_address = pd.Series(zip(B_NV.full_address, B_NV.neighborhoods)).apply(lambda x: lib.parse_address(*x))
B_address.set_index(B_NV.index, inplace=True)
col = B_address.columns.values
col[0] = 'city_'
B_address.columns = col
B_NV = pd.concat([B_NV, B_address], axis=1)
B_NV['name_'] = B_NV.name.apply(standard_name)

In [674]:
B_NV.tail().T

,61168,61169,61170,61175,61178
attributes,"{u'Take-out': True, u'Takes Reservations': Fal...","{u'Take-out': True, u'Accepts Credit Cards': T...","{u'Take-out': True, u'Accepts Credit Cards': T...","{u'Take-out': False, u'Price Range': 2, u'Outd...","{u'Take-out': False, u'Accepts Credit Cards': ..."
business_id,F14WDg0i6yZwXbflJ8ndaw,5jpV_QXxhfs8wHOXX3E_3w,m0yKevji9esIZdqmK3IVPQ,AG3jFaQcABSFF-ry3l5t5g,qrgod2lrYnxHwNWC_YYTyQ
categories,"[Pizza, Restaurants]","[Burgers, Fast Food, Sandwiches, Restaurants]","[Lebanese, Mediterranean, Middle Eastern, Rest...","[Breakfast & Brunch, Buffets, Restaurants]","[Seafood, American (New), Latin American, Rest..."
city,Las Vegas,Las Vegas,Henderson,Las Vegas,Las Vegas
full_address,Caesars Palace Hotel & Casino\n3570 Las Vegas ...,"7210 S Durango Dr\nSouthwest\nLas Vegas, NV 89113",10100 S Eastern Ave\nSte 120\nAnthem\nHenderso...,Excalibur Hotel Casino\n3850 Las Vegas Blvd So...,"2620 Regatta Dr\nSummerlin\nLas Vegas, NV 89128"
hours,{},"{u'Monday': {u'close': u'00:00', u'open': u'00...","{u'Monday': {u'close': u'21:00', u'open': u'10...","{u'Monday': {u'close': u'22:00', u'open': u'07...","{u'Monday': {u'close': u'22:00', u'open': u'11..."
latitude,36.1172,36.0572,36.0059,36.0989,36.2075
longitude,-115.178,-115.279,-115.112,-115.176,-115.268
name,Di Fara Pizza,Checkers Rally's,Shawarma Plus,The Buffet at Excalibur,Latin Fish
neighborhoods,[The Strip],"[Southwest, Spring Valley]",[Anthem],[The Strip],[Summerlin]


In [676]:
H = pd.read_csv('../data/vegas/restaurant_establishments.csv', delimiter=';', header=None, skiprows=1)
H.columns = ['permit_number', 'facility_id', 'PE', 'restaurant_name',  'location_name',
             'address', 'latitude', 'longitude', 'city_id', 
             'city_name', 'zip_code', 'nciaa', 'plan_review', 'record_status',
             'current_grade', 'current_demerits', 'date_current', 'previous_grade', 
             'date_previous', 'misc','empty']

In [462]:
H.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20467 entries, 0 to 20466
Data columns (total 21 columns):
permit_number       20467 non-null object
facility_id         20467 non-null object
PE                  20467 non-null int64
restaurant_name     20465 non-null object
location_name       20467 non-null object
address             20444 non-null object
latitude            20457 non-null float64
longitude           20457 non-null float64
city_id             20467 non-null int64
city_name           20467 non-null object
zip_code            20449 non-null object
nciaa               15160 non-null object
plan_review         13 non-null float64
record_status       20467 non-null int64
current_grade       19265 non-null object
current_demerits    19914 non-null float64
date_current        19265 non-null object
previous_grade      18177 non-null object
date_previous       18177 non-null object
misc                20467 non-null object
empty               0 non-null float64
dtypes: float6

In [470]:
H.head().T

,0,1,2,3,4
permit_number,PR0000002,PR0000003,PR0000004,PR0000005,PR0000006
facility_id,FA0002677,FA0002678,FA0002679,FA0002680,FA0002681
PE,1006,1008,1007,1007,1006
restaurant_name,McDonald's #3549 D HOTEL,McDonald's #5551,McDonald's #7675,McDonald's 2922,McDonald's #3153 Plaza
location_name,D HOTEL - McDonalds #3549,McDonalds Corporation,McDonalds #7675 Rancho & Cheyenne,McDonalds 2922,Plaza Hotel - McDonalds
address,301 Fremont St,4400 E Charleston Blvd,3175 N Rancho Dr,2020 N Rainbow Blvd,1 S Main St
latitude,36.1694,36.1594,36.2168,36.1964,36.1722
longitude,-115.143,-115.08,-115.212,-115.241,-115.147
city_id,10,10,10,10,10
city_name,Las Vegas,Las Vegas,Las Vegas,Las Vegas,Las Vegas


In [678]:
reload(lib)

<module 'merge_vegas' from 'merge_vegas.pyc'>

In [679]:
NV = H.copy()
NV['name_'] = H.restaurant_name.apply(lib.standard_name)
NV['city_'] = NV.city_name.apply(lambda x: '_'.join(x.lower().strip().split()))
NV['zip'] = NV.zip_code.apply(lambda x: np.nan if type(x)==float else x[:5])

NV = pd.concat([NV, NV.address.apply(lib.split_address)], axis=1)

NV.head().T

,0,1,2,3,4
permit_number,PR0000002,PR0000003,PR0000004,PR0000005,PR0000006
facility_id,FA0002677,FA0002678,FA0002679,FA0002680,FA0002681
PE,1006,1008,1007,1007,1006
restaurant_name,McDonald's #3549 D HOTEL,McDonald's #5551,McDonald's #7675,McDonald's 2922,McDonald's #3153 Plaza
location_name,D HOTEL - McDonalds #3549,McDonalds Corporation,McDonalds #7675 Rancho & Cheyenne,McDonalds 2922,Plaza Hotel - McDonalds
address,301 Fremont St,4400 E Charleston Blvd,3175 N Rancho Dr,2020 N Rainbow Blvd,1 S Main St
latitude,36.1694,36.1594,36.2168,36.1964,36.1722
longitude,-115.143,-115.08,-115.212,-115.241,-115.147
city_id,10,10,10,10,10
city_name,Las Vegas,Las Vegas,Las Vegas,Las Vegas,Las Vegas


In [680]:
NV_ind = ['name_','num','street','city_','zip']
#NV_ind = ['n' 'street','city_','zip']
temp = NV.set_index(NV_ind).join(B_NV.set_index(NV_ind), how='inner', rsuffix = 'y_')
print temp.shape

H_x = NV[~NV.facility_id.isin(temp.facility_id)]
B_x = B_NV[~B_NV.business_id.isin(temp.business_id)]

NV_ind2 = ['num','street','city_','zip']
temp2 = H_x.set_index(NV_ind2).join(B_x.set_index(NV_ind2), how='inner', rsuffix = 'y_')

print temp2.shape

temp_fuzz = lib.append_fuzz_scores(temp2)

(952, 40)
(23257, 42)


In [525]:
temp.head().T

name_                                                     applebees  \
num                                                           10305   
street                                             south eastern av   
city_                                                     henderson   
zip                                                           89052   
permit_number                                             PR0019126   
facility_id                                               FA0063678   
PE                                                             1003   
restaurant_name                         Applebee's #6013 Restaurant   
location_name                         Applebee's Restaurant and Bar   
address                                         10305 S Eastern Ave   
latitude                                                    36.0023   
longitude                                                  -115.109   
city_id                                                           7   
city_name                                                 Henderson   
zip_code                                                 89052-3960   
nciaa                                                             Y   
plan_review                                                     NaN   
record_status                                                     1   
current_grade                                                     A   
current_demerits                                                  6   
date_current                                    2014-01-21 00:00:00   
previous_grade                                                    A   
date_previous                                   2013-06-24 00:00:00   
misc              Applebee's #6013 Restaurant Applebees 6013 Res...   
empty                                                           NaN   
suite                                                                 
attributes        {u'Take-out': True, u'Accepts Credit Cards': T...   
business_id                                  Zzj67RkbWf9mcbotiCfDlA   
categories        [Bars, American (Traditional), Burgers, Nightl...   
city                                                      Henderson   
full_address       10305 S Eastern Ave\nAnthem\nHenderson, NV 89052   
hours                                                            {}   
latitudey_                                                   36.002   
longitudey_                                                -115.109   
name                                                     Applebee's   
neighborhoods                                              [Anthem]   
open                                                           True   
review_count                                                     46   
stars                                                           2.5   
state                                                            NV   
type                                                       business   
addressy_                                                       NaN   
complex                                                               
suitey_                                                               

name_                                                                \
num                                                                   
street                                                                
city_                                                                 
zip                                                           89052   
permit_number                                             PR0019127   
facility_id                                               FA0063678   
PE                                                             1016   
restaurant_name                                Applebee's #6013 Bar   
location_name                         Applebee's Restaurant and Bar   
address                                         10305 S Eastern Ave   
latitude               

In [681]:
ind_A = (temp_fuzz['max'] >= 75) & (temp_fuzz.avg_w_2 >= 75)
ind_B = (temp_fuzz['max'] >= 60) & (temp_fuzz.avg_w_2 >= 60) & (temp_fuzz.avg_w_3 >= 80)
NV_level2 = pd.concat([temp, temp_fuzz[ind_A | ind_B]])

print NV_level2.shape

H_x2 = NV[~NV.facility_id.isin(NV_level2.facility_id)]
B_x2 = B_NV[~B_NV.business_id.isin(NV_level2.business_id)]

(2965, 65)


In [682]:
NV_ind3 = ['city_','zip']
temp3 = H_x2.set_index(NV_ind3).join(B_x2.set_index(NV_ind3), how='inner', rsuffix = 'y_')

In [683]:
temp3.shape

(2536119, 46)

In [684]:
# For now only save level1 (exact matches)... 
#     casino names are causing too much blur when we try to do fuzzy name matching w/in the
#     same casino complex
NV_level1 = temp.copy()
save_to_pickle(NV_level1,'../data/vegas/vegas_yelp_merge.pkl')

In [529]:
temp2.to_csv('../data/vegas/merge_dump_01.csv', encoding='utf-8')

In [531]:
temp_fuzz.to_csv('../data/vegas/merge_dump_02.csv', encoding='utf-8')

### Charlotte, NC

In [402]:
def split_address(x):
    address = {'suite':''}
    x = x.lower().replace('\r','').replace('\n','')
    x = x.replace(' ste-',' ste ')
    
    words = x.split()
    # some street numbers have a dash, append the second part to suite
    if words[0].find('-') > 0:
        n = words[0].split('-')
        address['suite'] += n[-1] + ' '
        x = ' '.join([n[0]] + words[1:])
    
    x = x.replace('-',' ')
    x = re.sub('[%s]' % re.escape(string.punctuation.replace('&','')), '', x)
    abbr = {'road':'rd', 'street':'st', 'avenue':'av', 'ave':'av', 'drive':'dr', 'boulevard':'blvd',
            'lane':'ln', 'circle':'cir', 'building':'building', 'mount':'mt', 
            'n':'north', 'e':'east', 's':'south', 'w':'west', 'suite':'ste', 'bv':'blvd', 'suit':'ste',
            'pky':'pkwy', 'parkway':'pkwy',
            'first':'1st', 'second':'2nd', 'third':'3rd', 'fourth':'4th', 'fifth':'5th', 'sixth':'6th',
            'seventh':'7th', 'eighth':'8th', 'ninth':'9th', 'tenth':'10th'}
    for key, value in abbr.iteritems():
        x = re.sub(r'\b(%s)\b' % key, value, x) 
    x = x.strip()
    
    n = re.findall(r'^(\d+\D?)\b',x)
    s = re.search(r'\b(ste\W?\D?\W?\d*\W?\D?)|(unit\W?\D?\W?\d*\W?\D?)\Z', x)
    if len(n) > 0 and s is not None and len(s.group()) > 0:
        i = re.search(r'\b(ste)|(bldg)|(unit)\b', x)
        address['num'] = n[0]
        address['street'] = re.sub(r'^(\d+\D?)\b', '', x[:x.find(i.group())])
        address['suite'] += x[x.find(i.group()):]
    elif len(n) > 0:
        address['num'] = n[0]
        address['street'] = re.sub(r'^(\d+\D?)\b', '', x)
    else:
        address = {'num':'', 'street':x[0], 'suite':''}
        
    address = lib.strip_address_letters(address)
        
    # remove extra white spaces
    for key, value in address.iteritems():
        address[key] = re.sub(r'\s+', ' ', value).strip()
        
    return pd.Series(address)
    
def standard_name(x):
    rep_list = {'@':' at ', '-':' ', '/':' '}
    sub_list = {'grille':'grill', 'ristorante':'restaurant','restaurante':'restaurant',
                'italiano':'italian', 'mexicano':'mexican', 'mexicana':'mexican'}
    if type(x) == float:
        x = str(x)
    x = x.lower()
    for key, value in rep_list.iteritems():
        x = x.replace(key, value)
    for key, value in sub_list.iteritems():
        x = re.sub(r'\b(%s)\b' % key, value, x)
    
    x = x.strip()
    x = re.sub(r'(\#\d+)\Z','',x)
    x = re.split(r'\b(at)\b', x)[0].strip()
    return re.sub('[%s]' % re.escape(string.punctuation), '', x)

In [ ]:
B_NC = B[(B.state=='NC')& (B.categories.apply(lambda x: 'Restaurants' in x))]
B_NC['address'] = B_NC.full_address.apply(lambda x: x.replace('\n',' ')\
                                                     .replace('Ste','Suite')\
                                                     .replace(',','')\
                                                     .replace(' NC ', ' '))

In [ ]:
B_NC.city.value_counts()

In [6]:
H = open_pickle('../data/char/char_FULL_04.pkl')

In [ ]:
H.city.value_counts()

In [404]:
NC = pd.DataFrame.from_records(H).T
NC.head(1)

,address,city,county,inspections,name,state,type,zip
122131,1318 W MOREHEAD ST \r\n,CHARLOTTE,Mecklenburg,"[[2/26/2015, 90, A, 2398], [10/3/2014, 93, A, ...",OPEN KITCHEN,NC,1 - Restaurant,28208


In [405]:
NC['city_'] = NC.city.apply(lambda x: x.lower().strip())
NC['name_'] = NC.name.apply(standard_name)
NC['id_'] = NC.index

NC = pd.concat([NC, NC.address.apply(split_address)], axis=1)
NC[NC.id_==123786].head(1)

,address,city,county,inspections,name,state,type,zip,city_,name_,id_,num,street,suite
123786,516-A N GRAHAM ST \r\n,CHARLOTTE,Mecklenburg,"[[4/15/2015, 95, A, 2398], [11/19/2014, 91.50,...",SUB-ONE,NC,1 - Restaurant,28202,charlotte,sub one,123786,516,north graham st,a


In [406]:
B_address = pd.Series(zip(B_NC.full_address, B_NC.neighborhoods)).apply(lambda x: lib.parse_address(*x))
B_address.set_index(B_NC.index, inplace=True)
col = B_address.columns.values
col[0] = 'city_'
B_address.columns = col
B_NC = pd.concat([B_NC, B_address], axis=1)
B_NC['name_'] = B_NC.name.apply(standard_name)

In [242]:
B_NC[B_NC.business_id=='C3QfGqD3qjWUbwBOkEjqSQ'].full_address.tolist()

[u'9630 University City Blvd\nSte F\nUniversity City\nCharlotte, NC 28213']

In [407]:
NC_ind = ['name_','num','street','city_','zip']
temp = NC.set_index(NC_ind).join(B_NC.set_index(NC_ind), how='inner', rsuffix = 'y_')
temp.shape

(642, 27)

In [408]:
H_x = NC[~NC.id_.isin(temp.id_)]
B_x = B_NC[~B_NC.business_id.isin(temp.business_id)]

In [409]:
NC_ind2 = ['num','street','city_','zip']
temp2 = H_x.set_index(NC_ind2).join(B_x.set_index(NC_ind2), how='inner', rsuffix = 'y_')

In [410]:
temp2.shape

(1598, 29)

In [ ]:
temp2.to_csv('../data/char/merge_dump_01.csv', encoding='utf-8')

In [411]:
temp_fuzz = lib.append_fuzz_scores(temp2)

In [412]:
temp_fuzz.to_csv('../data/char/merge_dump_07.csv', encoding='utf-8')

In [216]:
reload(lib)

<module 'merge_charlotte' from 'merge_charlotte.py'>

In [413]:
ind_A = (temp_fuzz['max'] >= 75) & (temp_fuzz.avg_w_2 >= 75)
ind_B = (temp_fuzz['max'] >= 60) & (temp_fuzz.avg_w_2 >= 60) & (temp_fuzz.avg_w_3 >= 80)
NC_level2 = pd.concat([temp, temp_fuzz[ind_A | ind_B]])

In [414]:
NC_level2.shape

(1134, 52)

In [415]:
H_x2 = NC[~NC.id_.isin(NC_level2.id_)]
B_x2 = B_NC[~B_NC.business_id.isin(NC_level2.business_id)]

In [416]:
NC_ind3 = ['city_','zip']
temp3 = H_x2.set_index(NC_ind3).join(B_x2.set_index(NC_ind3), how='inner', rsuffix = 'y_')

In [417]:
temp3.shape

(54046, 33)

In [418]:
temp_fuzz3 = lib.append_fuzz_scores(temp3)

In [419]:
ind_A3 = (temp_fuzz3['max'] >= 75) & (temp_fuzz3.avg_w_2 >= 75)
ind_B3 = (temp_fuzz3['max'] >= 60) & (temp_fuzz3.avg_w_2 >= 60) & (temp_fuzz3.avg_w_3 >= 80)
temp_fuzz3[ind_A3 | ind_B3].to_csv('../data/char/merge_dump_12.csv', encoding='utf-8')

In [440]:
def to_int(x):
    try:
        out = int(x)
    except:
        out = np.nan
    return out

ind_A3 = (temp_fuzz3['max'] >= 75) & (temp_fuzz3.avg_w_2 >= 75)
ind_B3 = (temp_fuzz3['max'] >= 60) & (temp_fuzz3.avg_w_2 >= 60) & (temp_fuzz3.avg_w_3 >= 80)
ind_C3 = temp_fuzz3.num == temp_fuzz3.numy_
ind_D3 = (temp_fuzz3.street == temp_fuzz3.street) & \
         (abs(temp_fuzz3.num.apply(to_int) - temp_fuzz3.numy_.apply(to_int)) <= 100)
NC_level3 = pd.concat([NC_level2, temp_fuzz3[(ind_A3 | ind_B3) & (ind_C3 | ind_D3)]])

In [442]:
NC_level3.shape

(1332, 56)

In [443]:
H_x3 = NC[~NC.id_.isin(NC_level3.id_)]
B_x3 = B_NC[~B_NC.business_id.isin(NC_level3.business_id)]

In [445]:
save_to_pickle(NC_level3,'../data/char/charlotte_yelp_merge.pkl')

In [444]:
print H_x3.shape
print B_x3.shape

(1602, 14)
(764, 23)


In [284]:
reload(lib)

<module 'merge_charlotte' from 'merge_charlotte.py'>

In [430]:
b_list = [129641,129644,129560,151433,127252,128514,128700,124251,128677,130023,123415,
          126428,126920,160381,123786,124262,127957,144402]
bad_list = ['lmY05KHoqyFZjGsuHYx-HQ', 'SNqqdZNsC1w8mL15SHJEWw', 
            'WjVUvwDBosUunBm3uY-pyA', 'cE8IX3c8V47cBou08KsklA',
            'T1jYZFB_7cqdhuvzpxfFWQ', 'yvNnNl1DqVRmVuvCBytxuw',
            '4jl0_Ny3wForCqs06DRzRg', 'tZ1FVdbAna3lUL5LGcAa_Q',
            'cEtlUnEBXDPsFHhhou27tw', 'RguKChB4sdPwYA_1_S6JMg',
            'HKwlJatD3lZjzmF8n_9dNA', 'mH3fUPIdmh5ybrmD48pJkQ',
            '6ZtftjQ18WUN3-IW_yA3tQ', 'Iu0ElolxFlhCKimijFTPNw',
            'h2fqHWVr6jmjqpwr3gELzg', 'JF4Ge_KqznXoZ-zmk_veQw']
bad_ind = B_NC.business_id.isin(bad_list)
b_ind = NC.id_.isin(b_list)

In [431]:
pd.Series(zip(B_NC[bad_ind].full_address, B_NC[bad_ind].neighborhoods)).apply(lambda x: lib.parse_address(*x))

,city,complex,num,street,suite,zip
0,pineville,,9940,pineville matthews rd,,28134
1,matthews,,1905,matthews township pkwy,,28105
2,charlotte,,10223,university city blvd,,28213
3,charlotte,,1601,elizabeth av,,28204
4,charlotte,,8542,university city blvd,,28213
5,charlotte,charlotte douglas international airport,5501,r c josh birmingham pkwy,,28208
6,charlotte,,1100,metropolitan av,bldg east ste 100,28204
7,charlotte,,8332,pinevillematthews rd,ste 201203,28226
8,charlotte,,10405,park rd,b,28210
9,pineville,,9101,pineville matthews rd,,28134


In [432]:
B_NC[bad_ind][['full_address']]

,full_address
720,"9940 Pineville Matthews Rd\nPineville, NC 28134"
9214,"1905 Matthews Township Parkway\nMatthews, NC 2..."
9621,10223 University City Blvd\nUniversity City\nC...
22748,"1601 Elizabeth Ave\nElizabeth\nCharlotte, NC 2..."
26452,8542 University City Blvd\nUniversity City\nCh...
31718,Charlotte Douglas International Airport\n5501 ...
31798,"1100 Metropolitan Ave\nBldg E, Ste 100\nMyers ..."
31984,8332 Pineville-Matthews Rd\nSte 201-203\nCharl...
32788,"10405-B Park Rd\nCharlotte, NC 28210"
33087,"9101 Pineville Matthews Rd\nPineville, NC 28134"


In [428]:
NC[b_ind].address.apply(split_address)

,num,street,suite
123415,1600,matthews mint hill rd,
123786,516,north graham st,a
124251,4520,north tryon st,41
124262,710,west trade st,g
126428,2910,mt holly huntersville rd,
126920,9005,j m keynes blvd,2
127252,440,east mccullough dr,230
127957,265,mtholly huntersville rd,
128514,10405,park rd,
128677,15105,john j delaney dr,a


In [429]:
NC[b_ind][['address']]

,address
123415,1600 MATTHEWS-MINT HILL RD \r\n
123786,516-A N GRAHAM ST \r\n
124251,4520-41 N TRYON ST \r\n
124262,710-G W TRADE ST \r\n
126428,2910 MT. HOLLY-HUNTERSVILLE RD \r\n
126920,9005-2 J M KEYNES BLVD \r\n
127252,440-230 E MCCULLOUGH DR \r\n
127957,265 MT.HOLLY-HUNTERSVILLE RD \r\n
128514,10405 PARK RD \r\n
128677,15105-A JOHN J DELANEY DR \r\n


In [375]:
y = u'1100 Metropolitan Ave Bldg E Ste 100 Myers'.lower()
x = re.search(r'\b(ste)|(bldg)|(unit)\b',y)

In [322]:
re.split(r'\b%s\b' % 'Pineville', '9101 Pineville Matthews Rd\nPineville, NC 28134')

['9101 ', ' Matthews Rd\n', ', NC 28134']

In [315]:
y[y.find(x.group()):]

u'bldg e ste 100 myers'

In [329]:
x.group()

u'bldg'

## merge cities with final script

In [90]:
import merge_phoenix as lib
import merge_main as lib_main

In [91]:
reload(lib)
reload(lib_main)

<module 'merge_main' from 'merge_main.pyc'>

In [92]:
B2 = lib_main.preprocess_yelp(lib_main.get_yelp_businesses(), 'AZ')
H2 = lib.preprocess_AZ()

In [93]:
H2.head()

,address,cutting_edge,link,name,permit_id,has_i,has_v,city,complex,num,street,suite,zip,name_
0,120 N Valentine St Wickenburg 85358,,/EnvSvc/OnlineApplication/EnvironmentalHealth/...,Wickenburg Community Ctr,FD-00001,True,False,wickenburg,,120,north valentine st,,85358,wickenburg community ctr
1,11820 N 81st Ave Peoria 85345,,/EnvSvc/OnlineApplication/EnvironmentalHealth/...,Peoria Boys & Girls Club,FD-00002,True,False,peoria,,11820,north 81st av,,85345,peoria boys girls club
2,613 N 4th Ave Phoenix 85003,,/EnvSvc/OnlineApplication/EnvironmentalHealth/...,Phoenix Silvercrest,FD-00003,True,True,phoenix,,613,north 4th av,,85003,phoenix silvercrest
3,8561 N 61st Ave Glendale 85302,,/EnvSvc/OnlineApplication/EnvironmentalHealth/...,Ywca Senior Citizens,FD-00004,True,True,glendale,,8561,north 61st av,,85302,ywca senior citizens
4,7410 E Sutton Pl Scottsdale 85260,,/EnvSvc/OnlineApplication/EnvironmentalHealth/...,Thunderbird Academy,FD-00005,True,True,scottsdale,,7410,east sutton pl,,85260,thunderbird academy


In [94]:
B2.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,...,state,type,address,name_,city_,complex,num,street,suite,zip
1878,"{u'Take-out': True, u'Accepts Credit Cards': T...",x5Mv61CnZLohZWxfCVCPTQ,"[Sandwiches, Pizza, Chicken Wings, Restaurants]",Phoenix,"2819 N Central Ave\nPhoenix, AZ 85004",{},33.479542,-112.073418,Domino's Pizza,[],...,AZ,business,2819 N Central Ave Phoenix 85004,dominos pizza,phoenix,,2819,north central av,,85004
1879,"{u'Take-out': True, u'Noise Level': u'quiet', ...",2ZnCITVa0abGce4gZ6RhIw,"[American (New), Sandwiches, Restaurants]",Phoenix,"1850 N Central Ave\nPhoenix, AZ 85004",{},33.468988,-112.074315,Viad Tower Restaurants,[],...,AZ,business,1850 N Central Ave Phoenix 85004,viad tower restaurants,phoenix,,1850,north central av,,85004
1880,"{u'Alcohol': u'full_bar', u'Price Range': 1, u...",EmzaQR5hQlF0WIl24NxAZA,"[American (New), Nightlife, Dance Clubs, Resta...",Phoenix,"132 E Washington St\nPhoenix, AZ 85004","{u'Sunday': {u'close': u'02:00', u'open': u'21...",33.448399,-112.071702,Sky Lounge,[],...,AZ,business,132 E Washington St Phoenix 85004,sky lounge,phoenix,,132,east washington st,,85004
1883,"{u'Take-out': True, u'Accepts Credit Cards': T...",KPoTixdjoJxSqRSEApSAGg,"[Thai, Restaurants]",Phoenix,"2631 N Central Ave\nPhoenix, AZ 85004","{u'Monday': {u'close': u'21:00', u'open': u'11...",33.477934,-112.073524,Wild Thaiger,[],...,AZ,business,2631 N Central Ave Phoenix 85004,wild thaiger,phoenix,,2631,north central av,,85004
1891,"{u'Alcohol': u'full_bar', u'Open 24 Hours': Fa...",e5kc0CQ4R-PCCDgb274gSg,"[Mexican, Tex-Mex, Restaurants]",Phoenix,"455 N 3rd St\nSte 114\nPhoenix, AZ 85004","{u'Monday': {u'close': u'21:00', u'open': u'11...",33.452603,-112.069119,Canyon Cafe,[],...,AZ,business,455 N 3rd St Suite 114 Phoenix 85004,canyon cafe,phoenix,,455,north 3rd st,ste 114,85004


## Charlotte create features:

In [5]:
df_nc = open_pickle('../data/char/charlotte_yelp_features.pkl')

In [6]:
df_nc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6613 entries, 0 to 6743
Data columns (total 12 columns):
business_id    6613 non-null object
id_            6613 non-null int64
date           6613 non-null object
inspec_id      6613 non-null object
score          6613 non-null object
grade          6613 non-null object
date_start     0 non-null datetime64[ns]
rev_ct         6613 non-null int64
neg_ct         6613 non-null int64
stars_avg      6613 non-null float64
rev_len_avg    6613 non-null float64
stars_var      6613 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 671.6+ KB


In [7]:

NC = open_pickle('../data/char/charlotte_yelp_merge.pkl')

In [8]:
import features_charlotte as lib

In [25]:
reload(lib)

<module 'features_charlotte' from 'features_charlotte.pyc'>

In [30]:
%%time
df_NC = lib.get_features_NC(NC, '2011-06-30', 'charlotte', ['score','grade'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41485 entries, 0 to 41484
Data columns (total 9 columns):
_id            41485 non-null object
business_id    41485 non-null object
date           41485 non-null object
review_id      41485 non-null object
stars          41485 non-null int64
text           41485 non-null object
type           41485 non-null object
user_id        41485 non-null object
votes          41485 non-null object
dtypes: int64(1), object(8)
memory usage: 3.2+ MB
None
(241574, 15)
(1382, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6744 entries, 0 to 6743
Data columns (total 7 columns):
business_id    6744 non-null object
id_            6744 non-null int64
date           6744 non-null object
inspec_id      6744 non-null object
score          6744 non-null object
grade          6744 non-null object
date_start     6744 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 421.5+ KB
None
CPU times: user 58.3 s, sys: 2.76 s, tot

In [27]:
save_to_pickle(df_NC, '../data/char/charlotte_yelp_features.pkl')

In [13]:
I = lib.get_NC_inspections(NC)

In [14]:
import features_main as lib_main

In [22]:
reload(lib_main)

<module 'features_main' from 'features_main.py'>

In [23]:
I2 = lib_main.generate_inspec_dates(I)

In [24]:
I2.head()

,date_start
inspec_id,
168590_0,2015-04-02
168489_0,2015-04-14
168173_0,2015-03-11
168145_0,2015-04-02
167990_0,2015-03-13


In [21]:
I.head()

,date,score,grade,inspector,id_,inspec_id
0,3/3/2015,97,A,2219,161416,161416_0
1,12/5/2014,96,A,2219,161416,161416_1
2,9/24/2015,97,A,2427,127669,127669_0
3,5/26/2015,97.50,A,2447,127669,127669_1
4,1/15/2015,97,A,2447,127669,127669_2


In [29]:
NC.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1332 entries, (, 321, north caldwell st, charlotte, 28202) to (pineville, 28134)
Data columns (total 56 columns):
address                    1332 non-null object
addressy_                  1332 non-null object
attributes                 1332 non-null object
avg                        690 non-null float64
avg_2                      690 non-null float64
avg_3                      690 non-null float64
avg_w                      690 non-null float64
avg_w_2                    690 non-null float64
avg_w_3                    690 non-null float64
business_id                1332 non-null object
categories                 1332 non-null object
city                       1332 non-null object
cityy_                     1332 non-null object
complex                    1332 non-null object
county                     1332 non-null object
full_address               1332 non-null object
fuzz_partial_ratio         690 non-null float64
fuzz_partial_ratio_2     

In [28]:
df_NC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 6742
Data columns (total 12 columns):
business_id    284 non-null object
id_            284 non-null int64
date           284 non-null object
inspec_id      284 non-null object
score          284 non-null object
grade          284 non-null object
date_start     284 non-null datetime64[ns]
rev_ct         284 non-null int64
neg_ct         284 non-null int64
stars_avg      284 non-null float64
rev_len_avg    284 non-null float64
stars_var      284 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 28.8+ KB


# Pittsburgh, PA

In [7]:
import merge_pittsburgh as lib
import merge_main as lib_main

In [145]:
reload(lib)
reload(lib_main)

<module 'merge_main' from 'merge_main.py'>

In [41]:
H = open_pickle('../data/pitt/pitt_inspections_FINAL.pkl')

In [146]:
B_PA = lib_main.preprocess_yelp(B, 'PA')
PA = lib.preprocess_PA()

In [147]:
merge_level1 = lib_main.merge_exact_match(PA, B_PA)

(4290, 34)


In [148]:
merge_level2 = lib_main.merge_partial_match2(PA, B_PA, merge_level1, merge_level=2, dump_tag='02')

(7277, 36)
(7832, 59)


In [149]:
merge_level3 = lib_main.merge_partial_match2(PA, B_PA, merge_level2, merge_level=3, dump_tag='03')

(2732, 38)
(8447, 61)


In [150]:
merge_level4 = lib_main.merge_partial_match2(PA, B_PA, merge_level3, merge_level=4, dump_tag='04')

(27453, 38)
(8757, 63)


In [131]:
foo = "JD's McMillans JDF J Doug N.W.E.S LaFeria GF TRally TR"
foo = "J.D.'s McMillans J D F J Doug N. W. E.S LaFeria G F TRally T R"

x = foo
bar = re.findall(r'[a-z][A-Z]', x)

for w in bar:
    x = x.replace(w, w[0] + ' ' + w[1])
    
bar = re.findall(r'[A-Z]+[a-z]',x)
for w in bar:
    x = x.replace(w, w.lower())

bar = re.findall(r'(?=([A-Z]\.? [A-Z]))', x)

for w in bar:
    x = x.replace(w, w.replace(' ',''))
x

"J.D.'s mc millans JDFJ doug N.W.E.S la feria GF trally TR"

In [125]:
x = foo
bar = re.findall(r'[a-z][A-Z]', x)

for w in bar:
    x = x.replace(w, w[0] + ' ' + w[1])

print x
    
bar = re.findall(r'[A-Z]+[a-z]',x)
bar

JD's Mc Millans JDF J Doug N.W.E.S La Feria GF TRally TR


['Mc', 'Mi', 'Do', 'La', 'Fe', 'TRa']